In [1]:
import pandas as pd
from quantrl_lab.data import (
    DataSourceRegistry,
    DataProcessor,
    IndicatorRegistry,
    AlpacaDataLoader, 
    YfinanceDataloader, 
    AlphaVantageDataLoader
)

from quantrl_lab.data.indicators.technical_indicators import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

#### What each data source supports (as of the latest implementation)

In [2]:
data_source_apc = AlpacaDataLoader()
supported_features = data_source_apc.supported_features
print(f"Supported features for AlpacaDataLoader: {supported_features}") 

data_source_yf = YfinanceDataloader()
supported_features = data_source_yf.supported_features
print(f"Supported features for YfinanceDataLoader: {supported_features}")

data_source_av = AlphaVantageDataLoader()
supported_features = data_source_av.supported_features
print(f"Supported features for AlphaVantageDataLoader: {supported_features}")

Supported features for AlpacaDataLoader: ['historical_bars', 'news', 'live_data', 'streaming']
Supported features for YfinanceDataLoader: ['historical_bars']
Supported features for AlphaVantageDataLoader: ['historical_bars', 'news']


In [3]:
print(f"Available indicators:  {IndicatorRegistry.list_all()}")

Available indicators:  ['SMA', 'EMA', 'RSI', 'MACD', 'ATR', 'BB', 'STOCH', 'OBV']


In [4]:
data_loader = DataSourceRegistry()

In [5]:
olhcv_df = data_loader.get_historical_ohlcv_data(
    symbols="MU",
    start="2023-01-01",
    end="2025-01-01",
    timeframe="1d",
)

Fetching historical data for MU from 2023-01-01 to 2025-01-01 with timeframe 1d

In [6]:
news_df = data_loader.get_news_data(
    "MU",
    start="2023-01-01",
    end="2025-01-01",
)

Output()

✓ Total news items fetched: 1370

In [7]:
data_processor = DataProcessor(olhcv_data=olhcv_df, news_data=news_df)

In [8]:
# Define comprehensive indicator configuration with multiple window sizes
indicators = [
    # Simple Moving Average - multiple windows
    {"SMA": {"window": [5, 10, 20, 50]}},
    
    # Exponential Moving Average - multiple windows
    {"EMA": {"window": [5, 10, 20, 50]}},
    
    # Relative Strength Index - multiple windows
    {"RSI": {"window": [7, 14, 21, 28]}},
    
    # MACD - multiple parameter combinations
    {"MACD": [
        {"fast": 12, "slow": 26, "signal": 9},  # Standard MACD
        {"fast": 5, "slow": 35, "signal": 5},   # Fast MACD
        {"fast": 8, "slow": 21, "signal": 5},   # Custom MACD
        {"fast": 19, "slow": 39, "signal": 9}   # Slow MACD
    ]},
    
    # Average True Range - multiple windows
    {"ATR": {"window": [7, 14, 21, 28]}},
    
    # Bollinger Bands - multiple windows and standard deviations
    {"BB": [
        {"window": 10, "num_std": 2.0},
        {"window": 20, "num_std": 2.0},  # Standard BB
        {"window": 20, "num_std": 2.5},
        {"window": 50, "num_std": 2.0}
    ]},
    
    # Stochastic Oscillator - multiple parameter combinations
    {"STOCH": [
        {"k_window": 14, "d_window": 3, "smooth_k": 1},  # Fast Stochastic
        {"k_window": 14, "d_window": 3, "smooth_k": 3},  # Slow Stochastic
        {"k_window": 21, "d_window": 5, "smooth_k": 3}   # Custom Stochastic
    ]},
    
    # On-Balance Volume (no parameters needed)
    "OBV"
]

In [9]:
processed_data = data_processor.data_processing_pipeline(
    indicators=indicators,
    fillna_strategy="neutral"
)

Applying SMA with params: {'window': 5}

Applying SMA with params: {'window': 10}

Applying SMA with params: {'window': 20}

Applying SMA with params: {'window': 50}

Applying EMA with params: {'window': 5}

...

Applying STOCH with params: {'k_window': 14, 'd_window': 3, 'smooth_k': 3}

Applying STOCH with params: {'k_window': 21, 'd_window': 5, 'smooth_k': 3}

Applying OBV with params: {}

Device set to use cpu


✓ Sentiment analysis pipeline initialized with model: ProsusAI/finbert

In [10]:
processed_data.tail()

,Open,High,Low,Close,Volume,Trade_count,VWAP,SMA_5,SMA_10,SMA_20,SMA_50,EMA_5,EMA_10,EMA_20,EMA_50,RSI_7,RSI_14,RSI_21,RSI_28,MACD_line_12_26,MACD_signal_9,MACD_line_5_35,MACD_signal_5,MACD_line_8_21,MACD_line_19_39,ATR_7,ATR_14,ATR_21,ATR_28,BB_middle_10,BB_upper_10_2.0,BB_lower_10_2.0,BB_bandwidth_10,BB_middle_20,BB_upper_20_2.0,BB_lower_20_2.0,BB_bandwidth_20,BB_upper_20_2.5,BB_lower_20_2.5,BB_middle_50,BB_upper_50_2.0,BB_lower_50_2.0,BB_bandwidth_50,STOCH_%K_14_1,STOCH_%D_3,STOCH_%K_14_3,STOCH_%K_21_3,STOCH_%D_5,OBV,sentiment_score
448,89.580,89.70,88.23,89.28,13021257.0,127908.0,89.078836,92.022,97.9765,99.11325,102.5961,92.330162,95.569614,98.176645,100.654756,33.083627,38.389479,41.010496,42.603327,-2.660272,-1.035791,-7.452382,-2.288622,-3.768755,-2.039387,5.844385,5.247510,4.904103,4.731871,97.9765,114.507978,81.445022,0.337458,99.11325,111.050452,87.176048,0.301100,114.034753,84.191747,102.5961,114.654587,90.537613,0.235067,20.140351,22.198647,21.637427,21.637427,37.057512,302556227.0,0.916668
449,88.500,90.63,88.10,89.82,14506092.0,151662.0,89.573184,89.206,96.7530,98.51425,102.3061,91.493442,94.524230,97.380774,100.229863,34.703900,39.166576,41.531921,42.994962,-2.982348,-1.297985,-7.735628,-2.875539,-4.049373,-2.346762,5.370901,5.053402,4.791051,4.653233,96.7530,113.747467,79.758533,0.351296,98.51425,111.070059,85.958441,0.318629,114.209012,82.819488,102.3061,114.881713,89.730487,0.245843,22.035088,20.287228,21.286550,21.286550,27.026948,317062319.0,0.928226
450,89.000,89.49,87.58,88.63,19422607.0,155389.0,88.569961,89.514,95.7920,98.03575,101.8939,90.538961,93.452552,96.547367,99.774967,32.670109,38.028255,40.699490,42.334816,-3.295628,-1.569855,-8.101271,-3.357775,-4.322247,-2.657333,4.923630,4.852445,4.669572,4.567046,95.7920,113.485232,78.098768,0.369409,98.03575,111.348586,84.722914,0.339489,114.676795,81.394705,101.8939,114.886049,88.901751,0.255013,17.859649,20.978558,20.011696,20.011696,21.578837,297639712.0,0.861282
451,86.985,87.05,84.58,85.31,22022912.0,219056.0,85.685342,88.552,94.0730,97.40375,101.3591,88.795974,91.972088,95.477142,99.207713,27.436568,34.974227,38.442361,40.534167,-3.768362,-1.871001,-9.103690,-3.859793,-4.863830,-3.075586,4.798826,4.795128,4.640069,4.548580,94.0730,112.204483,75.941517,0.385477,97.40375,111.882765,82.924735,0.371624,115.502519,79.304981,101.3591,114.837213,87.880987,0.265948,6.210526,18.888889,15.368421,15.368421,19.248360,275616800.0,0.880526
452,85.440,86.40,84.09,84.16,18691546.0,181722.0,84.681952,87.440,91.6630,96.68425,100.8193,87.250649,90.551708,94.399319,98.617607,25.768403,33.956949,37.682291,39.924178,-4.187531,-2.191863,-9.885700,-4.339701,-5.299518,-3.475313,4.443279,4.617618,4.529113,4.468630,91.6630,107.699134,75.626866,0.349893,96.68425,112.308309,81.060191,0.403997,116.214324,77.154176,100.8193,114.847573,86.791027,0.278285,2.175439,14.709552,8.748538,8.748538,17.410526,256925254.0,0.861282
